## 1. 环境准备

In [18]:
# 导入必要的库
import os
import sys
import yaml
import json
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from loguru import logger

# 添加项目路径
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# 加载环境变量
load_dotenv()

print(f"项目根目录: {project_root}")
print(f"Python路径: {sys.executable}")

项目根目录: /data1/tenglin/Software/homework/my_chinese_medical_rag
Python路径: /data1/tenglin/anaconda3/envs/medical/bin/python


In [19]:
# 加载配置
with open('config.yaml', 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

print("配置加载成功")
print(f"\n向量模型: {config['embedding']['model_path']}")
print(f"LLM提供商: {config['llm']['provider']}")
print(f"LLM模型: {config['llm']['model_name']}")

配置加载成功

向量模型: /data1/tenglin/models/bge-base-zh-v1.5
LLM提供商: vllm
LLM模型: Qwen3-1.7B


In [20]:
from src.data_loader import CMedQA2Loader

# 初始化数据加载器
loader = CMedQA2Loader(config)
print("数据加载器初始化完成")

数据加载器初始化完成


In [21]:
# 加载问题数据
questions_df = loader.load_questions()
print(f"问题数据: {len(questions_df)} 条")
print("\n前5条问题:")
questions_df.head(10)

2025-12-26 03:21:52.391 | INFO     | src.data_loader:load_questions:28 - 加载问题数据: ./dataset/cMedQA2/question.csv
2025-12-26 03:21:52.725 | INFO     | src.data_loader:load_questions:31 - 成功加载 120000 条问题数据


问题数据: 120000 条

前5条问题:


,question_id,content
0,65102009,头痛恶心肌肉痛关节痛颈部淋巴结疼痛怎么回事啊
1,44275784,我怀孕37周，肠子一直呼噜呼噜叫感觉像是在里面灌水，上厕所拉稀和喷水一样，一天上厕所5次，对...
2,42163349,男，67岁，2天前突然出现右小腿（类似抽筋症状），现出现右小腿消肿，有指压痕，无，请问可能是...
3,67935540,怀孕前两个月照了CT和X光，来两次月经后怀孕了，怀孕期间有轻微盆腔积水吃了两瓶杏香兔耳片。请...
4,33429289,阴囊湿冷阳痿早泻请问用点什么药
5,28517513,发烧全身发烫怎么办？能不能开空调的啊？？开始是全身无力，吃了药好了点。晚上睡觉空调开起来的，...
6,24279043,怀孕30周了，照了彩超，宝宝脐带绕颈已周，头位？怀孕30周了，照了彩超，宝宝脐带绕颈已周，头...
7,46574274,脚孤拐外侧肿胀疼痛怎么办用个什么药？
8,56439498,问题描述:我射精的时候精子射出来的那一瞬间阴茎下面很痛有时候小便也有一点点痛但没射精时那么厉...
9,37489044,为什么抽完血后会出现头晕、四肢无力脸色发白冒汗等现象？、请问为什么我抽完血后会出现头晕、四肢...


In [22]:
# 加载答案数据
answers_df = loader.load_answers()
print(f"答案数据: {len(answers_df)} 条")
print("\n前5条答案:")
answers_df.head()

2025-12-26 03:21:52.750 | INFO     | src.data_loader:load_answers:37 - 加载答案数据: ./dataset/cMedQA2/answer.csv
2025-12-26 03:21:53.774 | INFO     | src.data_loader:load_answers:40 - 成功加载 226266 条答案数据


答案数据: 226266 条

前5条答案:


,ans_id,question_id,content
0,0,45619783,月经延迟十四天而且伴随恶心，头痛，乏力的现象，那么考虑怀孕的概率是非常大的，建议你去医院检查...
1,1,45619783,如果你的月经周期规律，有正常的性生活，未采取任何有效的避孕措施，此时的症状考虑有怀孕的可能。...
2,2,45619783,建议在性生活过后14天左右可以用怀孕试纸自我检测一下，一般怀孕试纸显示2条线的话是怀孕了的，...
3,3,26616465,头痛是临床上最为常见的临床症状之一，是人体对各种致痛因素所产生的主观感觉，属于疼痛的范畴。建...
4,4,26616465,头痛主要是由于头部的血管、神经、脑膜等对疼痛敏感的组织受到刺激引起的。由紧张、疲劳、饮酒等原...


In [23]:
# 加载训练集候选对
train_candidates = loader.load_candidates('train')
print(f"训练集候选对: {len(train_candidates)} 条")

# 统计正负样本
positive = sum(1 for _, _, label in train_candidates if label == 1)
negative = len(train_candidates) - positive
print(f"正样本: {positive}, 负样本: {negative}")
print(f"\n前5条候选对:")
for i, (q_id, a_id, label) in enumerate(train_candidates[:5]):
    print(f"{i+1}. Question ID: {q_id}, Answer ID: {a_id}, Label: {label}")

2025-12-26 03:21:53.788 | INFO     | src.data_loader:load_candidates:73 - 加载train候选对数据: ./dataset/cMedQA2/train_candidates.txt
2025-12-26 03:21:57.258 | INFO     | src.data_loader:load_candidates:92 - 成功加载 10000000 条候选对数据


训练集候选对: 10000000 条
正样本: 5000000, 负样本: 5000000

前5条候选对:
1. Question ID: 24731702, Answer ID: 11064, Label: 1
2. Question ID: 24731702, Answer ID: 163246, Label: 0
3. Question ID: 24731702, Answer ID: 11064, Label: 1
4. Question ID: 24731702, Answer ID: 106174, Label: 0
5. Question ID: 24731702, Answer ID: 11064, Label: 1


In [24]:
# 构建QA对
qa_pairs, seen_pairs = loader.build_qa_pairs('train')
print(f"训练集QA对: {len(qa_pairs)} 条")

# 查看示例
print("\n示例QA对:")
for i, qa in enumerate(qa_pairs[:3]):
    print(f"\n{i+1}. 问题: {qa['question']}")
    print(f"   答案: {qa['answer']}")

2025-12-26 03:21:59.089 | INFO     | src.data_loader:build_qa_pairs:106 - 构建train集QA对...
2025-12-26 03:21:59.090 | INFO     | src.data_loader:load_questions:28 - 加载问题数据: ./dataset/cMedQA2/question.csv
2025-12-26 03:21:59.421 | INFO     | src.data_loader:load_questions:31 - 成功加载 120000 条问题数据
2025-12-26 03:21:59.422 | INFO     | src.data_loader:load_answers:37 - 加载答案数据: ./dataset/cMedQA2/answer.csv
2025-12-26 03:22:00.422 | INFO     | src.data_loader:load_answers:40 - 成功加载 226266 条答案数据
2025-12-26 03:22:00.423 | INFO     | src.data_loader:load_candidates:73 - 加载train候选对数据: ./dataset/cMedQA2/train_candidates.txt
2025-12-26 03:22:03.980 | INFO     | src.data_loader:load_candidates:92 - 成功加载 10000000 条候选对数据
构建QA对: 100%|██████████| 10000000/10000000 [00:05<00:00, 1861558.63it/s]
2025-12-26 03:22:09.647 | INFO     | src.data_loader:build_qa_pairs:145 - 成功构建 188490 条QA对


训练集QA对: 188490 条

示例QA对:

1. 问题: 不是说做b超对宝宝不好吗？那怀孕检查是不？不是说做b超对宝宝不好吗？那怀孕检查是不是越少越好。无麻烦解答，谢谢。
   答案: B超属于超声波经常检查是不好的而且也没有必要经常检查的一般怀孕两个月检查一下怀孕五个月检查一下快出生时在检查就可以还有就是不舒服检查就可以的

2. 问题: 不是说做b超对宝宝不好吗？那怀孕检查是不？不是说做b超对宝宝不好吗？那怀孕检查是不是越少越好。无麻烦解答，谢谢。
   答案: b超切实有一定的辐射，而且小孩比较的娇嫩，容易受辐射影响发育。宝宝尽量不要做b超，但是在胎儿期有母体的保护，所以不要担心，有必要的话一定要做。

3. 问题: 腰酸并伴有小腹胀痛怎么回事啊，而且引导有少量的血？
   答案: 可能是妇科炎症引起的建议放置达克宁栓口服三金片和龙胆泻肝丸如过不放心可以做一个支原体化验


## 3. 文本分块测试

In [25]:
from src.text_splitter import ChineseTextSplitter

# 初始化分块器
splitter = ChineseTextSplitter(config)
print("文本分块器初始化完成")
print(f"分块大小: {config['chunking']['chunk_size']}")
print(f"重叠大小: {config['chunking']['chunk_overlap']}")

文本分块器初始化完成
分块大小: 1024
重叠大小: 64


In [26]:
# 测试分块
# 取少量QA对进行测试
test_qa_pairs = qa_pairs[:100]

# 构建测试知识库
test_kb = []
for qa in test_qa_pairs:
    test_kb.append({
        'id': f"{qa['question_id']}_{qa['answer_id']}",
        'content': qa['combined_text'],
        'type': 'qa_pair',
        'question': qa['question'],
        'answer': qa['answer']
    })

# 分块
chunks = splitter.split_documents(test_kb)
print(f"\n生成文本块数: {len(chunks)} 个")

# 查看示例
print("\n示例文本块:")
for i, chunk in enumerate(chunks[:3]):
    print(f"\n块 {i+1}:")
    print(f"ID: {chunk['chunk_id']}")
    print(f"内容长度: {len(chunk['content'])} 字符")
    print(f"内容预览: {chunk['content']}...")

2025-12-26 03:22:10.593 | INFO     | src.text_splitter:split_documents:42 - 开始分块 100 个文档...
2025-12-26 03:22:10.594 | INFO     | src.text_splitter:split_documents:68 - 分块完成，共生成 100 个文本块



生成文本块数: 100 个

示例文本块:

块 1:
ID: 24731702_11064_0
内容长度: 133 字符
内容预览: 问题：不是说做b超对宝宝不好吗？那怀孕检查是不？不是说做b超对宝宝不好吗？那怀孕检查是不是越少越好。无麻烦解答，谢谢。
答案：B超属于超声波经常检查是不好的而且也没有必要经常检查的一般怀孕两个月检查一下怀孕五个月检查一下快出生时在检查就可以还有就是不舒服检查就可以的...

块 2:
ID: 24731702_11063_0
内容长度: 134 字符
内容预览: 问题：不是说做b超对宝宝不好吗？那怀孕检查是不？不是说做b超对宝宝不好吗？那怀孕检查是不是越少越好。无麻烦解答，谢谢。
答案：b超切实有一定的辐射，而且小孩比较的娇嫩，容易受辐射影响发育。宝宝尽量不要做b超，但是在胎儿期有母体的保护，所以不要担心，有必要的话一定要做。...

块 3:
ID: 11995968_157313_0
内容长度: 76 字符
内容预览: 问题：腰酸并伴有小腹胀痛怎么回事啊，而且引导有少量的血？
答案：可能是妇科炎症引起的建议放置达克宁栓口服三金片和龙胆泻肝丸如过不放心可以做一个支原体化验...


## 4. 向量检索测试

In [27]:
from src.vector_store import VectorStore, Retriever

# 初始化向量数据库
print("正在初始化向量数据库...")
vector_store = VectorStore(config)
print("向量数据库初始化完成")

2025-12-26 03:22:10.601 | INFO     | src.vector_store:__init__:26 - 加载Embedding模型: /data1/tenglin/models/bge-base-zh-v1.5


正在初始化向量数据库...
向量数据库初始化完成


In [28]:
# 构建向量索引（使用测试数据）
print("正在构建向量索引...")
print("注意: 这一步可能需要几分钟时间")

vector_store.build_index(chunks)
print("向量索引构建完成")
print(f"索引维度: {vector_store.dimension}")
print(f"文档数量: {len(vector_store.documents)}")

2025-12-26 03:22:10.991 | INFO     | src.vector_store:build_index:62 - 开始构建向量索引，共 100 个文档...


正在构建向量索引...
注意: 这一步可能需要几分钟时间


Batches: 100%|██████████| 4/4 [00:00<00:00, 12.06it/s]
2025-12-26 03:22:11.330 | INFO     | src.vector_store:build_index:77 - 向量索引构建完成，维度: 768, 文档数: 100


向量索引构建完成
索引维度: 768
文档数量: 100


In [29]:
# 测试检索 - 调试版本
retriever = Retriever(vector_store, config)

# 测试查询
test_queries = [
    "感冒了怎么办？",
    "发烧需要吃什么药？",
    "咳嗽有痰怎么治疗？"
]

for query in test_queries:
    # 先直接调用 vector_store.search 看原始结果
    raw_results = vector_store.search(query, top_k=10)
    print(f"\n原始检索结果数: {len(raw_results)}")
    if raw_results:
        print(f"最高相似度: {raw_results[0]['score']:.4f}")
        print(f"最低相似度: {raw_results[-1]['score']:.4f}")
    
    # 再调用 retriever.retrieve 看过滤后结果
    results, contexts = retriever.retrieve(query, top_k=10)
    
    print(f"过滤后结果数: {len(results)} (阈值: {config['vector_store']['score_threshold']})")
    
    if results:
        print(f"\n检索到 {len(results)} 条结果:\n")
        for i, result in enumerate(results, 1):
            print(f"结果 {i} (相似度: {result['score']:.4f}):")
            print(f"{result['content'][:200]}...")
            print()
    else:
        print("\n警告: 所有结果都被阈值过滤了！")
        print("建议降低 config.yaml 中的 score_threshold 值")

Batches: 100%|██████████| 1/1 [00:00<00:00, 121.68it/s]



原始检索结果数: 10
最高相似度: 0.5530
最低相似度: 0.4342


Batches: 100%|██████████| 1/1 [00:00<00:00, 130.19it/s]


过滤后结果数: 10 (阈值: 0.3)

检索到 10 条结果:

结果 1 (相似度: 0.5530):
问题：最近出现流感的症状感觉目前流感的种类太多了都不知道什么感冒了请问如果是一过性的流感样症状吃什么药好？一过性流感样症状吃什么药？
答案：潜伏期一般为1～3天刚开始普通感冒就算不吃药7-10天可自愈轻者注意休息注意保暖大量饮水即可症状明显者可适当对症吃药减轻症状若是出现了肺炎、发热等症状必须去医院诊治...

结果 2 (相似度: 0.4985):
问题：头痛，恶心，想吐，浑身无力的感觉
答案：你这个表现是胃肠型感冒的情况，可能是因为你受凉，饮食不当，抵抗力下降，病菌感染造成的建议你应该是注意休息，避免受凉，多喝水，清淡饮食，少吃多餐，细嚼慢咽，可以应用感康，吗丁啉，阿莫西林治疗...

结果 3 (相似度: 0.4796):
问题：肺炎，支气管炎，咳嗽前段时间检查说是肺炎，输水一星期，然后转发支气管炎，医生说吃药控制，急不得，现在老是偶尔咳嗽，但反复发烧。不知道怎么办啊。
答案：气管炎是由于感染或非感染因素引起的气管、支气管粘膜炎性变化，引起粘液分泌增多，临床上以长期咳嗽、咯痰或伴有喘息为主要特征建议改善居室环境，不宜太热，太冷，空调、暖气要开放适当，并注意通过使用排风扇或开窗等适时调节温度，使室内空气保持新鲜，室内外...

结果 4 (相似度: 0.4792):
问题：小孩鼻炎好治吗打呼噜打呼噜，鼻子不透气如何治疗才好
答案：，建议你防止感冒，口服药抗过敏药如：酮替芬鼻舒适，严重加上激素。另外一定注意脱离过敏源，就是使你加重的一切因素。冬天和早晨最好带上口罩。避免冷空气接触。另外你找一下你家里是否有可能过敏的因素，尽可能避免接触，如香水，鲜花，油漆，蚊香，装修材料，杀虫或空气清新剂和可能过敏的衣物等。或你注意观察你接触有的东西后加重或诱发你今后就应该注意不...

结果 5 (相似度: 0.4756):
问题：头痛，恶心，想吐，浑身无力的感觉
答案：结合上述描述，这个情况一般多是胃肠型感冒导致的，与近期饮食不当或者受凉都有一定关系的。因此，建议平时尽量清淡饮食，注意保暖，可以服用些藿香正气水、香砂养胃丸治疗的，可以较好地健脾祛湿，醒脾开胃，缓解这个情况的，另外，建议测量一下体温，以免耽误病情治疗。...

结果 6 (相似度: 0.4593):
问题

Batches: 100%|██████████| 1/1 [00:00<00:00, 133.33it/s]


...

过滤后结果数: 10 (阈值: 0.3)

检索到 10 条结果:

结果 1 (相似度: 0.5625):
问题：最近出现流感的症状感觉目前流感的种类太多了都不知道什么感冒了请问如果是一过性的流感样症状吃什么药好？一过性流感样症状吃什么药？
答案：潜伏期一般为1～3天刚开始普通感冒就算不吃药7-10天可自愈轻者注意休息注意保暖大量饮水即可症状明显者可适当对症吃药减轻症状若是出现了肺炎、发热等症状必须去医院诊治...

结果 2 (相似度: 0.5072):
问题：肺炎，支气管炎，咳嗽前段时间检查说是肺炎，输水一星期，然后转发支气管炎，医生说吃药控制，急不得，现在老是偶尔咳嗽，但反复发烧。不知道怎么办啊。
答案：气管炎是由于感染或非感染因素引起的气管、支气管粘膜炎性变化，引起粘液分泌增多，临床上以长期咳嗽、咯痰或伴有喘息为主要特征建议改善居室环境，不宜太热，太冷，空调、暖气要开放适当，并注意通过使用排风扇或开窗等适时调节温度，使室内空气保持新鲜，室内外...

结果 3 (相似度: 0.5030):
问题：孩子发烧但精神还好的时候会不会抽风
答案：孩子发烧如果这个体温过高，超过了39度，发生惊厥的可能是增高的，如果不超过一般是不会出现的。现在积极的治疗这个发烧的情况，避免体温过高的情况出现，这样可以预防惊厥的情况出现，建议通过布洛芬颗粒等退烧药治疗就可以了。...

结果 4 (相似度: 0.4535):
问题：头痛，恶心，想吐，浑身无力的感觉
答案：的话。建议口服罗红霉素和VC片进行治疗多吃蛋白质类食物如肉、蛋类不宜过多。油腻食物，如煎、炸类的更要少食，而应以碳水化合物类食物为主，可喝些粥类，也需多进食些蔬菜和水果...

结果 5 (相似度: 0.4478):
问题：未采取措施有高血压47岁可否服用
答案：建议你清淡饮食，禁烟酒，低盐低脂，少食糖，保持良好的情绪，饮食起居有规律，适度参加体育锻炼，控制体重在正常范围内。如果有高血脂必须同时治疗。降压可以选择硝苯地平缓释片或倍他乐克。...

结果 6 (相似度: 0.4426):
问题：头痛，恶心，想吐，浑身无力的感觉
答案：你这个表现是胃肠型感冒的情况，可能是因为你受凉，饮食不当，抵抗力下降，病菌感染造成的建议你应该是注意休息，避免受凉，多喝水，清淡饮食，少吃多餐，细嚼慢咽，可以应用感康，吗丁啉，阿莫西林治疗.

Batches: 100%|██████████| 1/1 [00:00<00:00, 132.11it/s]



原始检索结果数: 10
最高相似度: 0.5619
最低相似度: 0.3778


Batches: 100%|██████████| 1/1 [00:00<00:00, 133.72it/s]

过滤后结果数: 10 (阈值: 0.3)

检索到 10 条结果:

结果 1 (相似度: 0.5619):
问题：肺炎，支气管炎，咳嗽前段时间检查说是肺炎，输水一星期，然后转发支气管炎，医生说吃药控制，急不得，现在老是偶尔咳嗽，但反复发烧。不知道怎么办啊。
答案：气管炎是由于感染或非感染因素引起的气管、支气管粘膜炎性变化，引起粘液分泌增多，临床上以长期咳嗽、咯痰或伴有喘息为主要特征建议改善居室环境，不宜太热，太冷，空调、暖气要开放适当，并注意通过使用排风扇或开窗等适时调节温度，使室内空气保持新鲜，室内外...

结果 2 (相似度: 0.4582):
问题：小孩鼻炎好治吗打呼噜打呼噜，鼻子不透气如何治疗才好
答案：，建议你防止感冒，口服药抗过敏药如：酮替芬鼻舒适，严重加上激素。另外一定注意脱离过敏源，就是使你加重的一切因素。冬天和早晨最好带上口罩。避免冷空气接触。另外你找一下你家里是否有可能过敏的因素，尽可能避免接触，如香水，鲜花，油漆，蚊香，装修材料，杀虫或空气清新剂和可能过敏的衣物等。或你注意观察你接触有的东西后加重或诱发你今后就应该注意不...

结果 3 (相似度: 0.4410):
问题：最近出现流感的症状感觉目前流感的种类太多了都不知道什么感冒了请问如果是一过性的流感样症状吃什么药好？一过性流感样症状吃什么药？
答案：潜伏期一般为1～3天刚开始普通感冒就算不吃药7-10天可自愈轻者注意休息注意保暖大量饮水即可症状明显者可适当对症吃药减轻症状若是出现了肺炎、发热等症状必须去医院诊治...

结果 4 (相似度: 0.4328):
问题：小孩鼻炎好治吗打呼噜打呼噜，鼻子不透气如何治疗才好
答案：鼻炎会出现鼻塞，流清水涕，鼻痒，喉部不适，咳嗽等症状建议抗组胺药敏奇和中成药辛芩颗粒口服，日常可佩戴口罩，多喝水饮食清淡避免上火。...

结果 5 (相似度: 0.4251):
问题：小孩鼻炎好治吗打呼噜打呼噜，鼻子不透气如何治疗才好
答案：鼻炎多因外感风寒，抵抗力差等因素诱发的，药物治疗上建议使用头孢氨苄胶囊和千百鼻炎片，或者滴鼻净进行治疗较好，要多吃瓜果蔬菜补充维生素，积极锻炼身体增加机体抵抗力，可经常错错鼻翼二侧的迎香穴有一定的缓解作用...

结果 6 (相似度: 0.4189):
问题：小孩鼻炎好治吗打呼噜打呼噜，鼻子不透气如何治疗才好
答案：目前物理疗

## 5. LLM生成测试

In [30]:
from src.llm import LLMFactory

# 创建LLM实例
print("正在初始化LLM...")
print(f"LLM提供商: {config['llm']['provider']}")

try:
    llm = LLMFactory.create_llm(config)
    print("LLM初始化成功")
except Exception as e:
    print(f"LLM初始化失败: {e}")
    print("\n提示: 请检查:")
    print("1. 如果使用通义千问，请确保设置了DASHSCOPE_API_KEY环境变量")
    print("2. 如果使用本地模型，请确保model_path配置正确")
    print("3. 如果使用vLLM，请确保vLLM服务器已启动")
    llm = None

正在初始化LLM...
LLM提供商: vllm


2025-12-26 03:22:21.653 | INFO     | src.llm:__init__:272 - 连接到vLLM服务器: http://127.0.0.1:8000/v1
2025-12-26 03:22:21.654 | INFO     | src.llm:__init__:273 - 使用模型: Qwen3-1.7B


<think>
好的，用户问我是谁。首先，我需要明确自己的身份。我是AI助手，由阿里巴巴集团开发，基于通义实验室的模型。我应该简明扼要地介绍自己，同时保持友好和专业的语气。

接下来，用户可能想知道我的功能和用途，比如能做什么，如何帮助他们。我需要提到我的主要功能，如回答问题、提供信息、创作内容等，但也要注意不要过于冗长。

另外，用户可能对我的技术背景感兴趣，比如基于大模型，但不需要太深入的技术细节，保持简洁。同时，要强调我的学习能力和持续改进，让用户知道我是在不断进步的。

还要考虑用户可能的深层需求，比如他们可能想了解我的使用场景，或者是否有特定的问题需要帮助。因此，回答中应包含一些开放性的问题，鼓励用户提出更多问题。

最后，保持回答的友好和积极，让用户感到欢迎，并愿意继续交流。需要避免使用复杂术语，确保信息清晰易懂。
</think>

我是阿里巴巴集团旗下的通义实验室研发的AI助手，名为通义千问。我是一个大型语言模型，旨在通过自然语言处理技术，帮助用户解答问题、提供信息、创作内容，并进行多轮对话交流。

我的主要功能包括：
1. **回答问题**：提供准确、详细的解答，涵盖各种知识领域。
2. **信息查询**：帮助用户获取实时或最新的信息。
3. **创作辅助**：协助撰写文章、故事、代码等。
4. **多轮对话**：通过对话理解上下文，保持交流的连贯性。

我不断学习和优化，以更好地满足您的需求。如果您有任何问题或需要帮助，欢迎随时告诉我！
LLM初始化成功


In [31]:
# 测试LLM生成
if llm:
    test_prompt = "什么是感冒？请简要说明。"
    
    print(f"测试提示: {test_prompt}")
    print("\n生成中...\n")
    
    response = llm.generate(test_prompt)
    print(f"回答: {response}")
else:
    print("跳过LLM测试（LLM未初始化）")

测试提示: 什么是感冒？请简要说明。

生成中...

None
什么是感冒？请简要说明。
ChatCompletion(id='chatcmpl-58f5e8e1921c413c964dd6a76b8edc11', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='<think>\n嗯，用户问的是“什么是感冒？请简要说明。”首先，我需要确定用户的需求。他们可能对感冒的基本概念不太了解，或者想快速了解感冒的定义和症状。接下来，我要确保回答准确且简明。\n\n感冒通常由病毒引起，比如鼻病毒，但有时候其他病毒也可能导致。症状包括鼻塞、流鼻涕、咳嗽、喉咙痛、发烧等。不过，用户可能想知道更详细的分类，比如普通感冒和流感的区别，但问题里没提到，所以可能不需要深入。\n\n要避免使用专业术语过多，保持语言通俗易懂。同时，要提醒用户感冒常见于季节变化，比如秋冬，但也可以在其他季节发生。还要注意是否需要提到预防措施，但用户只要求简要说明，所以可能不需要展开。\n\n需要检查是否有遗漏的重要点，比如感冒的传播方式，但用户可能不需要那么详细。最后，确保回答结构清晰，分点或段落说明，但用户要求简要，所以保持简洁。\n</think>\n\n感冒是由病毒感染引起的急性呼吸道疾病，主要表现为鼻塞、流鼻涕、咳嗽、喉咙痛、打喷嚏、发热等症状。它通常由鼻病毒等病毒引起，多发于秋冬季节，传染性强，通过飞沫或接触传播。感冒多为自限性疾病，一般1-2周可自愈，无需特殊治疗，但可适当休息和补充水分。', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[], reasoning_content=None), stop_reason=None)], created=1766737341, model='Qwen3-1.7B', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(complet

## 6. 完整RAG流程测试

In [32]:
from src.rag_system import MedicalRAGSystem

# 初始化RAG系统（使用已构建的向量数据库和LLM）
print("正在初始化RAG系统...")

try:
    rag_system = MedicalRAGSystem(config, vector_store=vector_store, llm=llm)
    print("RAG系统初始化成功")
except Exception as e:
    print(f"RAG系统初始化失败: {e}")
    rag_system = None

2025-12-26 03:22:29.702 | INFO     | src.rag_system:__init__:53 - 医疗RAG系统初始化完成


正在初始化RAG系统...
RAG系统初始化成功


In [33]:
# 测试单次查询
if rag_system:
    test_question = "感冒了怎么办？"
    
    print(f"问题: {test_question}")
    print("\n正在查询...\n")
    
    result = rag_system.query(test_question, use_history=False)
    
    print(f"\n回答:\n{result['answer']}")
    print(f"\n引用来源数: {result['num_sources']}")
    
    if result['sources']:
        print("\n相关来源:")
        for i, source in enumerate(result['sources'][:3], 1):
            print(f"\n[{i}] (相似度: {source['score']:.3f})")
            print(source['content'][:200] + "...")
else:
    print("跳过RAG测试（RAG系统未初始化）")

2025-12-26 03:22:29.747 | INFO     | src.rag_system:query:72 - 收到查询: 感冒了怎么办？


问题: 感冒了怎么办？

正在查询...



Batches: 100%|██████████| 1/1 [00:00<00:00, 83.95it/s]
2025-12-26 03:22:29.765 | INFO     | src.rag_system:query:76 - 检索到 5 条相关文档


你是一个专业的中文医疗问答助手。你需要根据提供的医疗知识库内容来回答用户的健康相关问题。

要求：
1. 基于提供的上下文信息进行回答
2. 如果上下文中没有相关信息，明确说明"根据现有信息无法回答"
3. 回答要专业、准确、易懂
4. 必要时引用信息来源
5. 提醒用户：本系统仅供参考，具体诊疗请咨询专业医生

参考上下文信息回答问题。

上下文信息：
[1] 问题：最近出现流感的症状感觉目前流感的种类太多了都不知道什么感冒了请问如果是一过性的流感样症状吃什么药好？一过性流感样症状吃什么药？
答案：潜伏期一般为1～3天刚开始普通感冒就算不吃药7-10天可自愈轻者注意休息注意保暖大量饮水即可症状明显者可适当对症吃药减轻症状若是出现了肺炎、发热等症状必须去医院诊治

[2] 问题：头痛，恶心，想吐，浑身无力的感觉
答案：你这个表现是胃肠型感冒的情况，可能是因为你受凉，饮食不当，抵抗力下降，病菌感染造成的建议你应该是注意休息，避免受凉，多喝水，清淡饮食，少吃多餐，细嚼慢咽，可以应用感康，吗丁啉，阿莫西林治疗

[3] 问题：肺炎，支气管炎，咳嗽前段时间检查说是肺炎，输水一星期，然后转发支气管炎，医生说吃药控制，急不得，现在老是偶尔咳嗽，但反复发烧。不知道怎么办啊。
答案：气管炎是由于感染或非感染因素引起的气管、支气管粘膜炎性变化，引起粘液分泌增多，临床上以长期咳嗽、咯痰或伴有喘息为主要特征建议改善居室环境，不宜太热，太冷，空调、暖气要开放适当，并注意通过使用排风扇或开窗等适时调节温度，使室内空气保持新鲜，室内外温差不致太大。同时，进出房间，要及时加减衣服，适应环境变化，防止引发。

[4] 问题：小孩鼻炎好治吗打呼噜打呼噜，鼻子不透气如何治疗才好
答案：，建议你防止感冒，口服药抗过敏药如：酮替芬鼻舒适，严重加上激素。另外一定注意脱离过敏源，就是使你加重的一切因素。冬天和早晨最好带上口罩。避免冷空气接触。另外你找一下你家里是否有可能过敏的因素，尽可能避免接触，如香水，鲜花，油漆，蚊香，装修材料，杀虫或空气清新剂和可能过敏的衣物等。或你注意观察你接触有的东西后加重或诱发你今后就应该注意不要接触。尽可能避免到郊外，植物花粉丰富的地方去。一定要去出过敏因素。或者你去做一个过敏源的检查。查清楚后避免接触或脱敏治疗才能从根本上控制。

[5] 问题：头痛，恶心，想吐，浑身无力的感觉

2025-12-26 03:22:44.403 | INFO     | src.rag_system:query:120 - 查询完成


ChatCompletion(id='chatcmpl-1eb2766599b64ff1b39f317ebb0890e8', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='<think>\n好的，用户问的是“感冒了怎么办？”，需要根据提供的上下文信息来回答。首先，我需要回顾一下提供的五个上下文中的相关内容。\n\n第一个上下文提到一过性的流感样症状，建议多喝水、休息，必要时对症用药，如果出现肺炎或发热要就医。第二个上下文讲的是胃肠型感冒，建议注意休息、饮食清淡，可以使用感康、吗丁啉等药物。第三个上下文是关于肺炎和支气管炎的，提到咳嗽和发烧，需要控制病情，但用户现在有偶尔咳嗽和反复发烧，可能需要继续用药。第四个上下文是小孩鼻炎，提到过敏源和避免接触，但用户的问题是关于感冒，可能不相关。第五个上下文再次提到胃肠型感冒，建议清淡饮食和使用藿香正气水等药物。\n\n用户的问题比较广泛，需要综合这些信息。感冒可能有不同类型，比如普通感冒、胃肠型感冒、肺炎等。根据上下文，普通感冒建议多休息、喝水，必要时用药。胃肠型感冒需要清淡饮食，可能用藿香正气水。如果症状严重，如高烧或持续咳嗽，需要就医。同时，注意保暖和避免受凉，保持室内空气流通。\n\n需要检查是否有遗漏的信息。比如，是否有提到特定药物，如阿莫西林，但上下文中的回答可能没有提到，所以可能需要避免推荐具体药物，而是建议一般治疗措施。另外，用户可能想知道一般处理步骤，比如休息、饮食、药物等。\n\n需要确保回答准确，引用上下文中的建议，比如多喝水、休息，必要时用药，以及何时就医。同时，提醒用户如果症状持续或加重应及时就医。注意不要推荐具体药物，除非上下文明确提到，否则应避免。\n</think>\n\n根据现有信息，感冒的处理建议如下：\n\n1. **一般感冒**  \n   - **多休息**：保证充足睡眠，避免劳累。  \n   - **多喝水**：保持身体水分，缓解喉咙干燥。  \n   - **清淡饮食**：避免辛辣、油腻食物，可适量食用粥类、蔬菜等。  \n   - **对症用药**：若症状较轻（如轻微咳嗽、喉咙痛），可适当服用缓解症状的药物（如对乙酰氨基酚、氯

In [34]:
# 测试多轮对话
if rag_system:
    print("测试多轮对话\n")
    print("="*60)
    
    # 清空历史
    rag_system.clear_history()
    
    # 第一轮
    q1 = "什么是感冒？"
    print(f"\n用户: {q1}")
    r1 = rag_system.query(q1, use_history=True)
    print(f"助手: {r1['answer'][:200]}...")
    
    # 第二轮
    q2 = "应该怎么治疗？"  # 基于上下文的问题
    print(f"\n用户: {q2}")
    r2 = rag_system.query(q2, use_history=True)
    print(f"助手: {r2['answer'][:200]}...")
    
    # 第三轮
    q3 = "需要注意什么？"  # 继续基于上下文
    print(f"\n用户: {q3}")
    r3 = rag_system.query(q3, use_history=True)
    print(f"助手: {r3['answer'][:200]}...")
    
    print("\n" + "="*60)
    print(f"对话历史长度: {len(rag_system.get_history())} 轮")
else:
    print("跳过多轮对话测试（RAG系统未初始化）")

2025-12-26 03:22:44.415 | INFO     | src.rag_system:clear_history:188 - 对话历史已清空
2025-12-26 03:22:44.416 | INFO     | src.rag_system:query:72 - 收到查询: 什么是感冒？


测试多轮对话


用户: 什么是感冒？


Batches: 100%|██████████| 1/1 [00:00<00:00, 105.09it/s]
2025-12-26 03:22:44.432 | INFO     | src.rag_system:query:76 - 检索到 5 条相关文档


你是一个专业的中文医疗问答助手。你需要根据提供的医疗知识库内容来回答用户的健康相关问题。

要求：
1. 基于提供的上下文信息进行回答
2. 如果上下文中没有相关信息，明确说明"根据现有信息无法回答"
3. 回答要专业、准确、易懂
4. 必要时引用信息来源
5. 提醒用户：本系统仅供参考，具体诊疗请咨询专业医生

参考上下文信息回答问题。

上下文信息：
[1] 问题：最近出现流感的症状感觉目前流感的种类太多了都不知道什么感冒了请问如果是一过性的流感样症状吃什么药好？一过性流感样症状吃什么药？
答案：潜伏期一般为1～3天刚开始普通感冒就算不吃药7-10天可自愈轻者注意休息注意保暖大量饮水即可症状明显者可适当对症吃药减轻症状若是出现了肺炎、发热等症状必须去医院诊治

[2] 问题：头痛，恶心，想吐，浑身无力的感觉
答案：你这个表现是胃肠型感冒的情况，可能是因为你受凉，饮食不当，抵抗力下降，病菌感染造成的建议你应该是注意休息，避免受凉，多喝水，清淡饮食，少吃多餐，细嚼慢咽，可以应用感康，吗丁啉，阿莫西林治疗

[3] 问题：小孩鼻炎好治吗打呼噜打呼噜，鼻子不透气如何治疗才好
答案：，建议你防止感冒，口服药抗过敏药如：酮替芬鼻舒适，严重加上激素。另外一定注意脱离过敏源，就是使你加重的一切因素。冬天和早晨最好带上口罩。避免冷空气接触。另外你找一下你家里是否有可能过敏的因素，尽可能避免接触，如香水，鲜花，油漆，蚊香，装修材料，杀虫或空气清新剂和可能过敏的衣物等。或你注意观察你接触有的东西后加重或诱发你今后就应该注意不要接触。尽可能避免到郊外，植物花粉丰富的地方去。一定要去出过敏因素。或者你去做一个过敏源的检查。查清楚后避免接触或脱敏治疗才能从根本上控制。

[4] 问题：小孩鼻炎好治吗打呼噜打呼噜，鼻子不透气如何治疗才好
答案：鼻炎会出现鼻塞，流清水涕，鼻痒，喉部不适，咳嗽等症状建议抗组胺药敏奇和中成药辛芩颗粒口服，日常可佩戴口罩，多喝水饮食清淡避免上火。

[5] 问题：头痛，恶心，想吐，浑身无力的感觉
答案：结合上述描述，这个情况一般多是胃肠型感冒导致的，与近期饮食不当或者受凉都有一定关系的。因此，建议平时尽量清淡饮食，注意保暖，可以服用些藿香正气水、香砂养胃丸治疗的，可以较好地健脾祛湿，醒脾开胃，缓解这个情况的，另外，建议测量一下体温，以免耽误病情治疗。


问题：什么

2025-12-26 03:22:50.569 | INFO     | src.rag_system:query:120 - 查询完成
2025-12-26 03:22:50.570 | INFO     | src.rag_system:query:72 - 收到查询: 应该怎么治疗？


...

ChatCompletion(id='chatcmpl-94349b3ca6194bdda24735ca4eb5fecf', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='<think>\n好的，用户问的是“感冒怎么治疗”，但根据提供的上下文信息，里面并没有直接提到感冒的治疗方法。我需要仔细检查所有相关的条目，看看有没有关于感冒的信息。\n\n首先看条目[1]到[5]，这些都是关于其他疾病的问题，比如腰椎问题、水肿、手脚冰凉、腰椎滑脱、宫颈糜烂、阴道炎等。这些内容都没有涉及感冒的定义或治疗方法。条目[1]提到的是中药治疗，但具体是针对腰椎退行性变化的，而不是感冒。条目[2]和[3]讨论的是浮肿和手脚冰凉，可能与感冒有关，但没有明确说明。条目[4]和[5]涉及的是其他妇科问题，也没有提到感冒。\n\n所以，用户的问题是关于感冒的，但上下文中没有相关信息。根据要求，如果无法回答，需要明确说明。因此，正确的回应是指出无法提供相关信息，并建议咨询专业医生。\n\n同时，要确保回答符合专业性和易懂性，提醒用户具体诊疗需咨询医生。不需要引用任何信息来源，因为上下文没有相关数据。总结来说，用户的问题超出当前上下文的范围，无法回答。\n</think>\n\n根据现有信息无法回答。  \n**原因**：上下文中未提供关于感冒的定义或治疗方法，仅涉及其他疾病（如腰椎问题、妇科疾病等）。若需了解感冒的治疗，建议参考专业医学资料或咨询医生。  \n\n**提示**：本系统仅供参考，具体诊疗请咨询专业医生。', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[], reasoning_content=None), stop_reason=None)], created=1766737370, model='Qwen3-1.7B', object='chat.completion', service_tier=None, system_fingerprint=None, usage=Completio

Batches: 100%|██████████| 1/1 [00:00<00:00, 110.13it/s]
2025-12-26 03:22:57.693 | INFO     | src.rag_system:query:76 - 检索到 5 条相关文档


你是一个专业的中文医疗问答助手。你需要根据提供的医疗知识库内容来回答用户的健康相关问题。

要求：
1. 基于提供的上下文信息进行回答
2. 如果上下文中没有相关信息，明确说明"根据现有信息无法回答"
3. 回答要专业、准确、易懂
4. 必要时引用信息来源
5. 提醒用户：本系统仅供参考，具体诊疗请咨询专业医生

对话历史:
用户: 什么是感冒？
助手: <think>
好的，我现在需要回答用户的问题：“什么是感冒？”根据提供的上下文信息，我需要仔细查看是否有相关内容。

首先，查看上下文信息中的各个条目。条目[1]提到流感的症状和治疗，但没有直接定义感冒。条目[2]和[5]讨论的是胃肠型感冒的症状和治疗，但同样没有定义感冒。条目[3]和[4]是关于小孩鼻炎的治疗，也不涉及感冒的定义。

接下来，我需要确认是否有任何上下文信息中提到感冒的定义。看起来所有条目都没有直接定义感冒。用户的问题是关于感冒的定义，而提供的资料中没有明确的定义。因此，根据要求，如果上下文中没有相关信息，需要明确说明无法回答。

此外，虽然条目[1]提到流感和普通感冒，但没有区分两者，也没有给出感冒的定义。因此，无法从现有信息中得出感冒的准确定义。用户可能需要更专业的医学定义，而当前资料不足以回答这个问题。

因此，正确的回应是指出根据现有信息无法回答，并建议咨询专业医生。
</think>

根据现有信息无法回答。  
**原因**：上下文中未提供感冒的定义或相关解释，仅涉及流感症状、胃肠型感冒及鼻炎治疗等内容。若需准确定义感冒，建议参考专业医学资料或咨询医生。

用户: 应该怎么治疗？
助手: <think>
好的，用户问的是“感冒怎么治疗”，但根据提供的上下文信息，里面并没有直接提到感冒的治疗方法。我需要仔细检查所有相关的条目，看看有没有关于感冒的信息。

首先看条目[1]到[5]，这些都是关于其他疾病的问题，比如腰椎问题、水肿、手脚冰凉、腰椎滑脱、宫颈糜烂、阴道炎等。这些内容都没有涉及感冒的定义或治疗方法。条目[1]提到的是中药治疗，但具体是针对腰椎退行性变化的，而不是感冒。条目[2]和[3]讨论的是浮肿和手脚冰凉，可能与感冒有关，但没有明确说明。条目[4]和[5]涉及的是其他妇科问题，也没有提到感冒。

所以，用户的问题是关于感冒的，但上下文中没有相关信息。根据要求，如果无法回答，需要

2025-12-26 03:23:02.320 | INFO     | src.rag_system:query:120 - 查询完成


ChatCompletion(id='chatcmpl-06496da97bb642c89602a64b281ba33c', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='<think>\n好的，用户问的是“需要注意什么？”，但根据提供的上下文信息，没有直接提到感冒相关的注意事项。我需要仔细检查所有上下文条目，看看是否有相关内容。\n\n首先，条目[1]到[5]都是关于流产、妇科疾病和其他健康问题的，没有涉及感冒的注意事项。用户的问题可能与之前的对话有关，但上下文中没有相关信息。因此，根据要求，如果无法回答，需要明确说明。\n\n另外，用户可能是在询问感冒的注意事项，但当前上下文没有相关信息。因此，正确的回应是指出无法回答，并建议咨询专业医生。需要确保回答专业、准确，并提醒用户具体诊疗需咨询医生，同时避免引用任何上下文信息。\n</think>\n\n根据现有信息无法回答。  \n**原因**：上下文中未提供关于感冒的注意事项或相关健康建议。若需了解感冒的预防或护理措施，建议参考专业医疗指南或咨询医生。  \n\n**提示**：本系统仅供参考，具体诊疗请咨询专业医生。', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[], reasoning_content=None), stop_reason=None)], created=1766737377, model='Qwen3-1.7B', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=211, prompt_tokens=1307, total_tokens=1518, completion_tokens_details=None, prompt_tokens_details=None), prompt_logprobs=None)
助手: <thin